In [1]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats

In [2]:
#/data2/wangxuedong/mhc_test_data/first_review/csvfiles/pos_prob_split_chr1_xaa.csv
def integrate_csv_chromosome(filespath):
    dic_file=dict()
    for i in range(22):
        dic_file["chr"+str(i+1)]=[]
    #print(dic_file)
    for filepath in filespath:
        if filepath.split("/")[-1]=="posmhc.csv":
            dic_file["chr6"].append(filepath)
        else:
            dic_file[filepath.split("/")[-1].split(".")[0].split("_")[3]].append(filepath)
    #print(dic_file)
    for chr_,file_paths in dic_file.items():
        merged_data = pd.DataFrame()
        for file in file_paths:
            data = pd.read_csv(file)
            merged_data = merged_data.append(data, ignore_index=True)
        merged_data_sorted = merged_data.sort_values(by=merged_data.columns[0])
        merged_folderpath="/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/"+chr_+".csv"
        merged_data_sorted = merged_data_sorted.drop_duplicates(subset=merged_data_sorted.columns[0])
        if chr_=="chr6":
            merged_data_ls=merged_data_sorted[merged_data_sorted[merged_data_sorted.columns[0]] < 29720403]
            merged_data_ls.to_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/chr6_partA.csv",index=False)
            merged_data_mid=merged_data_sorted[(merged_data_sorted[merged_data_sorted.columns[0]] >= 29720403) & (merged_data_sorted[merged_data_sorted.columns[0]] <= 33129982)]
            merged_data_mid.to_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/chr6_mhcpart.csv",index=False)
            merged_data_bt=merged_data_sorted[merged_data_sorted[merged_data_sorted.columns[0]] > 33129982 ]
            merged_data_bt.to_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/chr6_partC.csv",index=False)           
        else:
            merged_data_sorted.to_csv(merged_folderpath, index=False)
      
            
need_merge_filespath="/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles/"
filespath=[need_merge_filespath+file for file in os.listdir(need_merge_filespath)]
# filespath
integrate_csv_chromosome(filespath)
        

/tmp/ipykernel_1481998/3581816211.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data, ignore_index=True)
/tmp/ipykernel_1481998/3581816211.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data, ignore_index=True)
/tmp/ipykernel_1481998/3581816211.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data, ignore_index=True)
/tmp/ipykernel_1481998/3581816211.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(data, ignore_index=True)
/tmp/ipykernel_1481998/3581816211.py:17: FutureWarning: The fram

In [3]:
def getAveragePer1000(filepath,n,filename):
    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    averages = [np.mean(csv_df.loc[i:i+n-1, "P"]) for i in range(0, csv_df.shape[0], n)]
    np.savetxt('/data2/wangxuedong/mhc_test_data/first_review/similar_region_bin_1000/'+filename+'.csv', averages, delimiter = ',')
    return averages

In [4]:
arrforDistribution=[]
for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/"):
    for file in files:        
        fileprefix=file[file.index("chr"):file.index(".")]
        filepath=os.path.join(curDir,file)
        arrforDistribution+=getAveragePer1000(filepath=filepath,n=1000,filename=fileprefix)
        

In [5]:
#get mean of Distribution
mean_arr=np.mean(arrforDistribution)
#get standard deviation of Distribution
std_arr=np.std(arrforDistribution)
print("mean_arr",mean_arr)
print("std_arr",std_arr)

mean_arr 0.008131350120614151
std_arr 0.003374720423931657


In [6]:
def compareWith0_01Pvalue(mean_arr,std_arr,filename,filepath):
    arr=[]
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    for i in range(len(csv_matrix)):
        mid_res=stats.norm.cdf(csv_matrix[i], mean_arr, std_arr)

        if mid_res>0.95:
            arr.append(1)
        else:
            arr.append(0)
    np.savetxt("/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_compare_with_01pvalue/"+filename+".csv", arr, delimiter = ',')
    return filename

In [7]:
for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/first_review/similar_region_bin_1000/"):
    for file in files:            
            filename=file[:file.index(".")]
            prefix=filename.split("_")[0]       
            filepath=os.path.join(curDir,file)
            compareWith0_01Pvalue(mean_arr=mean_arr,std_arr=std_arr,filename=filename,filepath=filepath)

In [8]:
def generatedfandcsv(columns,dic,filename):

    row_names=list(dic.keys())
    a=[]
    bs=[]
    #cs=[]
    for x in row_names:
        a.append(x)
        bs.append(dic[x])
        #cs.append(dic[x][1])
    dictforDF=dict()
    dictforDF[columns[0]]=a
    dictforDF[columns[1]]=bs
    #dictforDF[columns[2]]=cs

    df_index=pd.DataFrame(dictforDF)
    df_index.to_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvs_start_length/"+filename+"_start_lenth.csv")
    return df_index


In [9]:
def checkContinuous1(arr):
    res=dict()
    i=0
    while i<len(arr)-1:
        if arr[i]==1:

            count=1
            for j in range(i+1,len(arr),1):
                if arr[j]==1:
                    count+=1                    
                else:                  
                    break
            res[i]=count
            i+=count
        else:
            i+=1
            continue
    return res
            


In [10]:
def getstartAndLength(filepath,filename):
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    res=checkContinuous1(csv_matrix)
    df=generatedfandcsv(columns=["start","length"],dic=res,filename=filename)
    return df

In [11]:
for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_compare_with_01pvalue/"):
    for file in files:
        filename=file[:file.index(".")]
        filepath=os.path.join(curDir,file)
        getstartAndLength(filepath=filepath,filename=filename)

In [12]:
#Read a file ending with "start_length", and count how many kinds of 'length' and corresponding counts
def countNumofLength(dic,filepath):
    csv_data = pd.read_csv(filepath, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr=np.array(csv_df['length'])

    for i in range(len(arr)):

        if arr[i] in dic:

            dic[arr[i]]+=1
        else:
            dic[arr[i]]=1

    return dic

In [13]:
#Read each file ending with start_length, and count how many kinds of 'length' and corresponding counts
mydic=dict()
mhcdic=dict()
for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvs_start_length/"):
    for file in files:
            filename=file[:file.index(".")]
            if filename!="chr6_mhcpart_start_lenth":                  
                filepath=os.path.join(curDir,file)
                countNumofLength(mydic,filepath=filepath)
            else:
                 filepath_mhc=os.path.join(curDir,file)
                 countNumofLength(mhcdic,filepath=filepath_mhc)

In [14]:
mhcdic

{1: 5, 3: 1, 2: 2, 10: 1, 4: 1}

In [15]:
mydic

{1: 1945, 2: 463, 5: 12, 3: 146, 4: 42, 6: 7, 7: 6, 9: 1}

In [16]:
arr1=[]

for k2,arr in mydic.items():
    for i in range(arr):
        arr1.append(k2)

for k2,arr in mhcdic.items():
    for i in range(arr):
        arr1.append(k2)

In [17]:
mean_1=np.mean(arr1)
dev1=np.std(arr1)
print("mean",mean_1)
print("dev",dev1)
mid_res=stats.norm.ppf(0.9999999999,mean_1,dev1)
mid_res

mean 1.3890577507598785
dev 0.8199648000131434


6.6051333611960565

In [18]:
#third part
#get all of positions from vcf files
def getAllPositions(filepath):
    pos=[]
    df=pd.read_csv(filepath)
    pos=df["Unnamed: 0"].to_list()    
    return pos

In [19]:
def findPosition(filepath,start,length):
    arr=[]
    allpos=getAllPositions(filepath=filepath)
    arr.append(allpos[start*1000])
    if (start+length)*1000-1<len(allpos):
        end=allpos[(start+length)*1000-1]
    else:
        end=allpos[-1]
    arr.append(end)
    return arr



In [20]:
#Get the real position of corresponding to consecutive '1' in split_chr...
def GetPosition(filepath_start_length,res,threshold):
    csv_data = pd.read_csv(filepath_start_length, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]

    name=filepath_start_length.split("/")[-1].split(".")[0].split("_")[0]
    name1=filepath_start_length.split("/")[-1].split(".")[0].split("_")[1]
    if name=="chr6":

        if name1!="mhcpart":
            filepath="/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/"+name+"_"+name1+".csv"
    else:
        filepath="/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/"+name+".csv"
    for k,v in mydic.items():
        if v>=threshold:
            print("filepath",filepath)
            midarr.append(findPosition(filepath=filepath,start=k,length=v))
            
        else:
            continue
    if midarr!=[]:
        if name=="chr6":
            if name1!="mhcpart":
                res[name+"_"+name1]=midarr
        else:
            res[name]=midarr
    return res

In [21]:
#Get the real position of corresponding to consecutive '1' in mhc region
def getPositionforMHC(res):
    mhc_start_length_path="/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvs_start_length/chr6_mhcpart_start_lenth.csv"  #gai
    csv_data = pd.read_csv(mhc_start_length_path, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    arr_start=np.array(csv_df['start'])
    arr_length=np.array(csv_df['length'])
    mydic=dict()
    midarr=[]
    for i in range(len(arr_start)):
        mydic[arr_start[i]]=arr_length[i]
    for k,v in mydic.items():
        if v>=7:
            midarr.append(findPosition(filepath="/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/chr6_mhcpart.csv",start=k,length=v))            
        else:
            continue
    res["MHC"]=midarr
    return res
            



In [22]:
res=dict()
mhcres=dict()
for curDir, dirs, files in os.walk(top="/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvs_start_length/"):
    for file in files:
            filename=file[:file.index(".")]
            if filename!="chr6_mhcpart_start_lenth":
                filepath=os.path.join(curDir,file)
                GetPosition(filepath_start_length=filepath,res=res,threshold=7)
            else:
                getPositionforMHC(res=mhcres)

filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/chr2.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/chr3.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/chr9.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/chr11.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/chr12.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/chr12.csv
filepath /data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/chr14.csv


In [23]:
res

{'chr2': [[108377615, 108667023]],
 'chr3': [[87292011, 87555666]],
 'chr9': [[88434984, 88700396]],
 'chr11': [[39680311, 39930646]],
 'chr12': [[20836055, 21097074], [34328866, 34516639]],
 'chr14': [[31627550, 31888106]]}

In [24]:
mhcres

{'MHC': [[32589647, 32751796]]}